In [1]:
import tests
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from importlib import reload
import pandas as pd
import datasetBuilder
import tools
import scipy
from sklearn.ensemble import HistGradientBoostingClassifier as hgbc
from sklearn.ensemble import GradientBoostingClassifier as gbc
from sklearn.ensemble import RandomForestClassifier as rfc
import sklearn.base
import pickle
import copy
import tests
from sklearn.metrics import roc_auc_score as auc


In [12]:
target = datasetBuilder.get_target_df('/Users/jonahpoczobutt/projects/raw_data/nist_out.MSP').iloc[:,1:]

In [41]:
def auc_to_df(aucs, trues):

    mets =set()
    for i in aucs.iloc[0]:
        mets.add(i)

    mets=list(mets)
    outdict=dict()
    outdict['metric']=list()
    outdict['auc']=list()

    for metric in mets:
        temp = list()
        for i in range(len(aucs)):

            temp.append(aucs.iloc[i][metric])

        try:
            outdict['auc'].append(auc(trues, temp))
            outdict['metric'].append(metric)
        except:
            print(f'error on {metric}')
            

    return pd.DataFrame(outdict)
    

def create_variable_comparisons(target, size, ppm_threshes, noise_threshes, centroid_threshes, centroid_types, powers, sim_methods, max_matches,outfolder):

    for i in ppm_threshes:

        matches = datasetBuilder.create_matches_df(target,i,max_matches,size)
        filepath = f'{outfolder}/matches_{i}_ppm.csv'

        for j in noise_threshes:
            for k in range(len(centroid_threshes)):
                for l in powers:

                    cleaned = matches.apply(lambda x: datasetBuilder.clean_and_spec_features(x['query'],
                                                                                             x['query_prec'],
                                                                                             x['target'],
                                                                                             x['target_prec'],
                                                                                             noise_thresh=j,
                                                                                             centroid_thresh = centroid_threshes[k],
                                                                                             centroid_type=centroid_types[k],
                                                                                             power=l
                                                                                             ), 
                                                                                             axis=1,
                                                                                             result_type='expand')
                    
                    
                    cleaned=pd.DataFrame(cleaned)
                    cleaned = cleaned.iloc[:,-2:]
                    cleaned.columns = ['query','library']
                                                                             
                    aucs = tests.run_metrics_models_auc(sim_methods,[],cleaned, tol_thresh = centroid_threshes[k], tol_type=centroid_types[k])
                    aucs = auc_to_df(aucs, list(matches.iloc[:,-1]))
                    aucs['clean_specs'] =  f'{j}_{centroid_threshes[k]}_{centroid_types[k]}_{l}'
                    print(f'{j}_{centroid_threshes[k]}_{centroid_types[k]}_{l}')
                    aucs.to_csv(filepath, mode='a', header=False)

        

In [42]:
reload(tests)
matches=datasetBuilder.create_matches_df(target,10,100,1e2)
a = list(matches.columns)
a[-2]='library'
matches.columns=a

sim = tests.run_metrics_models_auc(None, [], matches, 5, 'ppm')

In [44]:
noise_threshes=[0.01,0.05,0.1]
centroid_tolerance_vals = [0.05,3]
centroid_tolerance_types=['da','ppm']
powers=[0.25,1,3,'ent',None]
ppm_threshes = [3,5,10,15]

create_variable_comparisons(target,
                            size=5e4,
                            ppm_threshes=ppm_threshes,
                            noise_threshes=noise_threshes,
                            centroid_threshes=centroid_tolerance_vals,
                            centroid_types=centroid_tolerance_types,
                            powers=powers,
                            sim_methods=None,
                            outfolder = '/Users/jonahpoczobutt/projects/specsim_res/individual_sims'
                            )



error on max_jensenshannon
error on max_correlation
0.01_0.05_da_0.25
error on max_jensenshannon
error on max_correlation
0.01_0.05_da_1
error on max_jensenshannon
error on max_correlation
0.01_0.05_da_3
error on max_jensenshannon
error on max_correlation
0.01_0.05_da_ent
error on max_jensenshannon
error on max_correlation
0.01_0.05_da_None
error on max_jensenshannon
error on max_correlation
0.01_3_ppm_0.25
error on max_jensenshannon
error on max_correlation
0.01_3_ppm_1
error on max_jensenshannon
error on max_correlation
0.01_3_ppm_3
error on max_jensenshannon
error on max_correlation
0.01_3_ppm_ent
error on max_jensenshannon
error on max_correlation
0.01_3_ppm_None
error on correlation
error on max_jensenshannon
error on max_correlation
0.05_0.05_da_0.25
error on correlation
error on max_jensenshannon
error on max_correlation
0.05_0.05_da_1
error on correlation
error on max_jensenshannon
error on max_correlation
0.05_0.05_da_3
error on correlation
error on max_jensenshannon
error on 

In [3]:
first_bases = np.load('/Volumes/Untitled/raw_data/first_bases.npy')
second_bases = np.load('/Volumes/Untitled/raw_data/second_bases.npy')

first_target = target[np.isin(target['inchi_base'],first_bases)]
#second_target = target[np.isin(target['inchi_base'],second_bases)]

first_target.reset_index(inplace=True)
#second_target.reset_index(inplace=True)


In [6]:
noise_threshes=[0.01,0.1,0.2]
centroid_tolerance_vals = [0.05]
centroid_tolerance_types=['da']
powers=[0.25,1,3]
sim_methods = ['lorentzian','squared_chord','lorentzian_jonah','entropy']

matches = datasetBuilder.create_matches_df(first_target,10,100,1e6)
#datasetBuilder.add_noises_to_matches(matches, scale_ratio=0.5, mult=200)
#del(target)
#del(first_target)

100007 rows created
200005 rows created
300026 rows created
400023 rows created
500033 rows created
600002 rows created
700078 rows created
800090 rows created
900038 rows created
1000088 rows created


In [7]:
del(target)
del(first_target)

In [10]:
reload(datasetBuilder)
datasetBuilder.chunk_create_all_to_all('/Users/jonahpoczobutt/projects/specsim_res/all_to_all_1.csv',matches, sim_methods = sim_methods, noise_threshes_query=noise_threshes, centroid_tolerance_vals_query=centroid_tolerance_vals, centroid_tolerance_types_query=centroid_tolerance_types,powers_query=powers,noise_threshes_target=noise_threshes, centroid_tolerance_vals_target=centroid_tolerance_vals, centroid_tolerance_types_target=centroid_tolerance_types,powers_target=powers)
del(matches)

processed 10000 rows
processed 110000 rows
processed 210000 rows
processed 310000 rows
processed 410000 rows
processed 510000 rows
processed 610000 rows
processed 710000 rows
processed 810000 rows
processed 910000 rows


In [13]:
second_target = target[np.isin(target['inchi_base'],second_bases)]
second_target.reset_index(inplace=True)

noise_threshes=[0.01,0.1,0.2]
centroid_tolerance_vals = [0.05]
centroid_tolerance_types=['da']
powers=[0.25,1,3]
sim_methods = ['lorentzian','squared_chord','lorentzian_jonah','entropy']

matches = datasetBuilder.create_matches_df(second_target,10,100,3e5)

datasetBuilder.chunk_create_all_to_all('/Users/jonahpoczobutt/projects/specsim_res/all_to_all_2.csv',matches, sim_methods = sim_methods, noise_threshes_query=noise_threshes, centroid_tolerance_vals_query=centroid_tolerance_vals, centroid_tolerance_types_query=centroid_tolerance_types,powers_query=powers,noise_threshes_target=noise_threshes, centroid_tolerance_vals_target=centroid_tolerance_vals, centroid_tolerance_types_target=centroid_tolerance_types,powers_target=powers)
del(matches)

100032 rows created
200005 rows created
300011 rows created
processed 10000 rows
processed 110000 rows
processed 210000 rows


In [15]:
target = datasetBuilder.get_target_df('/Volumes/Untitled/raw_data/nist_out.MSP').iloc[:,1:]

#first_bases = np.load('/Volumes/Untitled/raw_data/first_bases.npy')
#second_bases = np.load('/Volumes/Untitled/raw_data/second_bases.npy')

#first_target = target[np.isin(target['inchi_base'],first_bases)]
second_target = target[np.isin(target['inchi_base'],second_bases)]

#first_target.reset_index(inplace=True)
second_target.reset_index(inplace=True)

matches = datasetBuilder.create_matches_df(second_target,10,100,1e6)
datasetBuilder.add_noises_to_matches(matches, scale_ratio=0.5, mult=200)
del(target)
del(second_target)

FileNotFoundError: [Errno 2] No such file or directory: '/Volumes/Untitled/raw_data/nist_out.MSP'

In [ ]:
datasetBuilder.chunk_create_all_to_all('/Users/jonahpoczobutt/projects/specsim_res/all_to_all2_1_200.csv',matches, sim_methods = sim_methods, noise_threshes_query=noise_threshes, centroid_tolerance_vals_query=centroid_tolerance_vals, centroid_tolerance_types_query=centroid_tolerance_types,powers_query=powers,noise_threshes_target=noise_threshes, centroid_tolerance_vals_target=centroid_tolerance_vals, centroid_tolerance_types_target=centroid_tolerance_types,powers_target=powers)
del(matches)

processed 10000 rows
processed 110000 rows
processed 210000 rows
processed 310000 rows
processed 410000 rows
processed 510000 rows
processed 610000 rows
processed 710000 rows
processed 810000 rows
processed 910000 rows


In [10]:
del(matches)
del(mod)
del (first_target)
second_target = target[np.isin(target['inchi_base'],second_bases)]

noise_threshes=[0.01,0.1,0.2]
centroid_tolerance_vals = [0.05,1]
centroid_tolerance_types=['da','ppm']
powers=[0.25,1,3,'ent']
sim_methods = ['lorentzian','max_entropy','squared_chord','lorentzian_jonah','rbf','ave_bhattacharya_2','max_bhattacharya_2']

matches = datasetBuilder.create_matches_df(second_target,10,100,1e6)
datasetBuilder.add_noises_to_matches(matches, scale_ratio=0.5, mult=200)
print(f'match len predrop {len(matches)}')
matches=matches.dropna(how='any')
print(f'match len postdrop {len(matches)}')

mod = datasetBuilder.create_model_dataset(matches, sim_methods = sim_methods, noise_threshes=noise_threshes, centroid_tolerance_vals=centroid_tolerance_vals, centroid_tolerance_types=centroid_tolerance_types,powers=powers)
mod.to_csv('C:\\Users\\jonah\\projects\\specsim_res\\noise_second_0.5_200.csv')
del(matches)
del(mod)

NameError: name 'matches' is not defined

In [2]:
noise_threshes=[0.01,0.1,0.2]
centroid_tolerance_vals = [0.05]
centroid_tolerance_types=['da']
powers=[0.25,1,3]
sim_methods = ['lorentzian','squared_chord','lorentzian_jonah','entropy']

noise_threshes=[True, True, True]
centroid_tolerance_vals = [True]
centroid_tolerance_types=[True]
powers=[True,True, True]
sim_methods = [True, True, True, True]


def generate_keep_indices_all_to_all(noise_threshes_query, 
                                     centroid_tolerance_vals_query,
                                     powers_query, 
                                     noise_threshes_target, 
                                     centroid_tolerance_vals_target,
                                     powers_target,
                                     spec_features_query, 
                                     spec_features_target, 
                                     sim_methods, 
                                     any_=False, 
                                     nonspecs=False, 
                                     init_spec=False):
    
        if nonspecs:
            keep_indices= list(range(14))

        else:
            keep_indices=list()

        if init_spec:
            keep_indices+=list(range(14,24))

        ind=24
        for i in noise_threshes_target:
            for j in centroid_tolerance_vals_target:
                for k in powers_target:
                    
                    for l in spec_features_target:
                        if any_:
                            if True in [i,j,k,l,]:
                                keep_indices.append(ind)
                        else:
                            if i==j==k==l==True:
                                keep_indices.append(ind)
                        ind+=1
                        

        for i_ in noise_threshes_query:
            for j_ in centroid_tolerance_vals_query:
                for k_ in powers_query:

                    for l in spec_features_query:
                        if any_:
                            if True in [l,i_,j_,k_]:
                                keep_indices.append(ind)
                        else:
                            if l==i_==j_==k_==True:
                                keep_indices.append(ind)
                        
                        ind+=1
                        


        for i in noise_threshes_target:
            for j in centroid_tolerance_vals_target:
                for k in powers_target:
                    for i_ in noise_threshes_query:
                        for j_ in centroid_tolerance_vals_query:
                            for k_ in powers_query:

                                for l in sim_methods:
                                    
                                    if any_:
                                        if True in [i,j,k,l,i_,j_,k_]:
                                            keep_indices.append(ind)
                                    else:
                                        if i==j==k==l==i_==j_==k_==True:
                                            keep_indices.append(ind)
                                    
                                    ind+=1

        return keep_indices


In [24]:
spec_features_query=[True for i in range(4)]#8
spec_features_target=spec_features_query


generate_keep_indices_all_to_all(noise_threshes, 
                                centroid_tolerance_vals,
                                powers, 
                                noise_threshes, 
                                centroid_tolerance_vals,
                                powers,
                                spec_features_query=spec_features_query, 
                                spec_features_target=spec_features_target, 
                                sim_methods=sim_methods, 
                                any_=False, 
                                nonspecs=False, 
                                init_spec=False)

[24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185,
 186,
 187,
 188,
 189,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 2

In [3]:
train = pd.read_csv('/Users/jonahpoczobutt/projects/specsim_res/all_to_all_1.csv')
test = pd.read_csv('/Users/jonahpoczobutt/projects/specsim_res/all_to_all_2.csv')



In [5]:
spec_features_query=[True for i in range(4)]#8
spec_features_target=spec_features_query

noise_threshes=[True, True, True]
powers=[True,True, True]
noise_threshes_target=[True, True, True]
powers_target=[True,True, True]
sim_methods = [True, True, True, True]

all_columns=dict()

all_columns['all'] = generate_keep_indices_all_to_all(noise_threshes, 
                                centroid_tolerance_vals,
                                powers, 
                                noise_threshes_target, 
                                centroid_tolerance_vals,
                                powers_target,
                                spec_features_query=spec_features_query, 
                                spec_features_target=spec_features_target, 
                                sim_methods=sim_methods, 
                                any_=True, 
                                nonspecs=True, 
                                init_spec=True)

all_columns['all_but_nonspec'] = generate_keep_indices_all_to_all(noise_threshes, 
                                centroid_tolerance_vals,
                                powers, 
                                noise_threshes_target, 
                                centroid_tolerance_vals,
                                powers_target,
                                spec_features_query=spec_features_query, 
                                spec_features_target=spec_features_target, 
                                sim_methods=sim_methods, 
                                any_=False, 
                                nonspecs=False, 
                                init_spec=True)

all_columns['all_but_nonspec_and_init'] = generate_keep_indices_all_to_all(noise_threshes, 
                                centroid_tolerance_vals,
                                powers, 
                                noise_threshes_target, 
                                centroid_tolerance_vals,
                                powers_target,
                                spec_features_query=spec_features_query, 
                                spec_features_target=spec_features_target, 
                                sim_methods=sim_methods, 
                                any_=False, 
                                nonspecs=False, 
                                init_spec=False)

noise_threshes=[True, True, False]
powers=[True,True, False]

all_columns['23_query_setting'] = generate_keep_indices_all_to_all(noise_threshes, 
                                centroid_tolerance_vals,
                                powers, 
                                noise_threshes_target, 
                                centroid_tolerance_vals,
                                powers_target,
                                spec_features_query=spec_features_query, 
                                spec_features_target=spec_features_target, 
                                sim_methods=sim_methods, 
                                any_=False, 
                                nonspecs=False, 
                                init_spec=False)

noise_threshes=[True, False, False]
powers=[True,False, False]

all_columns['13_query_setting'] = generate_keep_indices_all_to_all(noise_threshes, 
                                centroid_tolerance_vals,
                                powers, 
                                noise_threshes_target, 
                                centroid_tolerance_vals,
                                powers_target,
                                spec_features_query=spec_features_query, 
                                spec_features_target=spec_features_target, 
                                sim_methods=sim_methods, 
                                any_=False, 
                                nonspecs=False, 
                                init_spec=False)

noise_threshes=[False, True, False]
powers=[False,True, False]

all_columns['13_query_setting_2'] = generate_keep_indices_all_to_all(noise_threshes, 
                                centroid_tolerance_vals,
                                powers, 
                                noise_threshes_target, 
                                centroid_tolerance_vals,
                                powers_target,
                                spec_features_query=spec_features_query, 
                                spec_features_target=spec_features_target, 
                                sim_methods=sim_methods, 
                                any_=False, 
                                nonspecs=False, 
                                init_spec=False)

noise_threshes=[True, False, False]
powers=[True,False, True]

all_columns['1_noise_barbell_powers'] = generate_keep_indices_all_to_all(noise_threshes, 
                                centroid_tolerance_vals,
                                powers, 
                                noise_threshes_target, 
                                centroid_tolerance_vals,
                                powers_target,
                                spec_features_query=spec_features_query, 
                                spec_features_target=spec_features_target, 
                                sim_methods=sim_methods, 
                                any_=False, 
                                nonspecs=False, 
                                init_spec=False)

noise_threshes=[True, False, True]
powers=[False,True, False]

all_columns['1_power_barbell_noise'] = generate_keep_indices_all_to_all(noise_threshes, 
                                centroid_tolerance_vals,
                                powers, 
                                noise_threshes_target, 
                                centroid_tolerance_vals,
                                powers_target,
                                spec_features_query=spec_features_query, 
                                spec_features_target=spec_features_target, 
                                sim_methods=sim_methods, 
                                any_=False, 
                                nonspecs=False, 
                                init_spec=False)

noise_threshes=[True, True, True]
powers=[True,True, True]
sim_methods = [True, False,False,False]

all_columns['all_loronly'] = generate_keep_indices_all_to_all(noise_threshes, 
                                centroid_tolerance_vals,
                                powers, 
                                noise_threshes_target, 
                                centroid_tolerance_vals,
                                powers_target,
                                spec_features_query=spec_features_query, 
                                spec_features_target=spec_features_target, 
                                sim_methods=sim_methods, 
                                any_=True, 
                                nonspecs=True, 
                                init_spec=True)

all_columns['all_but_nonspec_loronly'] = generate_keep_indices_all_to_all(noise_threshes, 
                                centroid_tolerance_vals,
                                powers, 
                                noise_threshes_target, 
                                centroid_tolerance_vals,
                                powers_target,
                                spec_features_query=spec_features_query, 
                                spec_features_target=spec_features_target, 
                                sim_methods=sim_methods, 
                                any_=False, 
                                nonspecs=False, 
                                init_spec=True)

all_columns['all_but_nonspec_and_init_loronly'] = generate_keep_indices_all_to_all(noise_threshes, 
                                centroid_tolerance_vals,
                                powers, 
                                noise_threshes_target, 
                                centroid_tolerance_vals,
                                powers_target,
                                spec_features_query=spec_features_query, 
                                spec_features_target=spec_features_target, 
                                sim_methods=sim_methods, 
                                any_=False, 
                                nonspecs=False, 
                                init_spec=False)


noise_threshes=[True, True, False]
powers=[True,True, False]

all_columns['23_query_setting_loronly'] = generate_keep_indices_all_to_all(noise_threshes, 
                                centroid_tolerance_vals,
                                powers, 
                                noise_threshes_target, 
                                centroid_tolerance_vals,
                                powers_target,
                                spec_features_query=spec_features_query, 
                                spec_features_target=spec_features_target, 
                                sim_methods=sim_methods, 
                                any_=False, 
                                nonspecs=False, 
                                init_spec=False)

noise_threshes=[True, False, False]
powers=[True,False, False]

all_columns['13_query_setting_loronly'] = generate_keep_indices_all_to_all(noise_threshes, 
                                centroid_tolerance_vals,
                                powers, 
                                noise_threshes_target, 
                                centroid_tolerance_vals,
                                powers_target,
                                spec_features_query=spec_features_query, 
                                spec_features_target=spec_features_target, 
                                sim_methods=sim_methods, 
                                any_=False, 
                                nonspecs=False, 
                                init_spec=False)

noise_threshes=[False, True, False]
powers=[False,True, False]

all_columns['13_query_setting_2_loronly'] = generate_keep_indices_all_to_all(noise_threshes, 
                                centroid_tolerance_vals,
                                powers, 
                                noise_threshes_target, 
                                centroid_tolerance_vals,
                                powers_target,
                                spec_features_query=spec_features_query, 
                                spec_features_target=spec_features_target, 
                                sim_methods=sim_methods, 
                                any_=False, 
                                nonspecs=False, 
                                init_spec=False)

noise_threshes=[True, False, False]
powers=[True,False, True]

all_columns['1_noise_barbell_powers_loronly'] = generate_keep_indices_all_to_all(noise_threshes, 
                                centroid_tolerance_vals,
                                powers, 
                                noise_threshes_target, 
                                centroid_tolerance_vals,
                                powers_target,
                                spec_features_query=spec_features_query, 
                                spec_features_target=spec_features_target, 
                                sim_methods=sim_methods, 
                                any_=False, 
                                nonspecs=False, 
                                init_spec=False)

noise_threshes=[True, False, True]
powers=[False,True, False]
sim_methods = [True, False,False,False]

all_columns['1_power_barbell_noise_loronly'] = generate_keep_indices_all_to_all(noise_threshes, 
                                centroid_tolerance_vals,
                                powers, 
                                noise_threshes_target, 
                                centroid_tolerance_vals,
                                powers_target,
                                spec_features_query=spec_features_query, 
                                spec_features_target=spec_features_target, 
                                sim_methods=sim_methods, 
                                any_=False, 
                                nonspecs=False, 
                                init_spec=False)




In [9]:
models = [
        hgbc(),
        hgbc(learning_rate=0.01),
        hgbc(learning_rate=0.5),
        hgbc(max_iter=200),
        hgbc(learning_rate=0.01, max_iter=200),
        hgbc(learning_rate=0.5, max_iter=200),
        ]
res_dict_a2a = best_models_by_subset(all_columns,train_sizes=[None],models=models, train=train, val=test[:50000], test=test[50000:],  )

finished all for None
finished all_but_nonspec for None
finished all_but_nonspec_and_init for None
finished 23_query_setting for None
finished 13_query_setting for None
finished 13_query_setting_2 for None
finished 1_noise_barbell_powers for None
finished 1_power_barbell_noise for None
finished all_loronly for None
finished all_but_nonspec_loronly for None
finished all_but_nonspec_and_init_loronly for None
finished 23_query_setting_loronly for None
finished 13_query_setting_loronly for None
finished 13_query_setting_2_loronly for None
finished 1_noise_barbell_powers_loronly for None
finished 1_power_barbell_noise_loronly for None


In [10]:
res_dict_a2a

{'all': [('HistGradientBoostingClassifier(learning_rate=0.5, max_iter=200)',
   0.2252,
   array([0.23556, 0.2438 , 0.24076, 0.22888, 0.23712, 0.22854]))],
 'all_but_nonspec': [('HistGradientBoostingClassifier()',
   0.28142,
   array([0.29554, 0.31032, 0.30218, 0.29862, 0.30242, 0.29862]))],
 'all_but_nonspec_and_init': [('HistGradientBoostingClassifier()',
   0.313156,
   array([0.3148 , 0.32762, 0.33644, 0.32078, 0.32252, 0.32662]))],
 '23_query_setting': [('HistGradientBoostingClassifier()',
   0.311456,
   array([0.31582, 0.32798, 0.33474, 0.31896, 0.32156, 0.3314 ]))],
 '13_query_setting': [('HistGradientBoostingClassifier()',
   0.3134,
   array([0.31684, 0.3274 , 0.32754, 0.31758, 0.32298, 0.31988]))],
 '13_query_setting_2': [('HistGradientBoostingClassifier(max_iter=200)',
   0.325808,
   array([0.33354, 0.33354, 0.33806, 0.32972, 0.33368, 0.33848]))],
 '1_noise_barbell_powers': [('HistGradientBoostingClassifier(max_iter=200)',
   0.313312,
   array([0.32008, 0.32626, 0.3267 ,

In [ ]:
def generate_keep_indices(noise_threshes, centroid_tolerance_vals, powers, spec_features, spec_change_features, sim_methods, any_=False, nonspecs=False, init_spec=False):

    if nonspecs:
        keep_indices= list(range(14))
    else:
        keep_indices=list()

    if init_spec:
        keep_indices+=list(range(14,24))

    ind=24
    for i in noise_threshes:
        for j in centroid_tolerance_vals:
            for k in powers:
                
                for l in spec_features:
                    if any_:
                        if True in [i,j,k,l]:
                            keep_indices.append(ind)
                    else:
                        if i==j==k==l==True:
                            keep_indices.append(ind)
                    ind+=1

                for l in spec_change_features:
                    if any_:
                        if True in [i,j,k,l]:
                            keep_indices.append(ind)
                    else:
                        if i==j==k==l==True:
                            keep_indices.append(ind)
                    ind+=1


                for l in sim_methods:
                    
                    if any_:
                        if True in [i,j,k,l]:
                            keep_indices.append(ind)
                    else:
                        if i==j==k==l==True:
                            keep_indices.append(ind)
                    ind+=1

    return keep_indices


In [ ]:
train.columns[14:24]

In [ ]:
noise_threshes=[0.01,0.1,0.2]
centroid_tolerance_vals = [0.05,1]
centroid_tolerance_types=['da','ppm']
powers=[0.25,1,3,'ent']
sim_methods = ['lorentzian','max_entropy','squared_chord','lorentzian_jonah','rbf','ave_bhattacharya_2','max_bhattacharya_2']


noise_threshes=[True,True,True]
centroid_tolerance_values=[True,False]
powers=[True,True,True,True]
spec_features=[True,True, False,False,False,False,False,False]#8
spec_change_features = [False for i in range(8)]#8
sim_methods=[True,False,False,False,False,False]#7

#cols['nonspec_only']=generate_keep_indices(noise_threshes, centroid_tolerance_values, powers, spec_features, spec_change_features, sim_methods, any_=True, nonspecs=True)
cols2['first_2_spec_lor_only']=generate_keep_indices(noise_threshes, centroid_tolerance_values, powers, spec_features, spec_change_features, sim_methods, any_=False, nonspecs=False, init_spec=True)



In [ ]:
with open('/Users/jonahpoczobutt/projects/specsim_res/col_dict.pickle', 'rb') as handle:
    cols = pickle.load(handle)

In [ ]:
cols_auc_test = [
    ('all',cols['all'],hgbc()),
    ('all_but_nonspec',cols['all_but_nonspec'],hgbc()),
    ('nonspec_only',cols['nonspec_only'],hgbc()),
    ('first_2_spec_lor_only',cols2['first_2_spec_lor_only'],hgbc()),
    ('first_2_spec',cols2['first_2_spec'],hgbc())
]

for i in cols_auc_test:
    i[2].fit(pd.concat((train.iloc[:,i[1]],val.iloc[:,i[1]])), pd.concat((train.iloc[:,-1:],val.iloc[:,-1:])))

    print(f'trained: {i[0]}')



In [ ]:
metrics=list()

for i in cols['all_sims']:

    metrics.append((test.columns[i], i))
    

In [ ]:
reload(tests)

aucs_res = tests.run_metrics_models_auc(metrics, cols_auc_test, test)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

a=np.random.beta(a=1,b=5,size=10000)*200
plt.hist(a, weights=np.ones(len(a)) / len(a), bins=40)
plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
plt.title('Distribution Over Peak Heights When Y=100')
plt.show()

In [ ]:
aucs_res

In [ ]:
aucs_res2

In [ ]:
from collections import Counter
mods = list()

for i in res_dict:

    mods=mods+[j[0] for j in res_dict[i]]

for i in res_dict2:

    mods=mods+[j[0] for j in res_dict2[i]]

Counter(mods)

In [ ]:
train = pd.read_csv('/Volumes/Untitled/specsim_res/noise_first_1_100.csv').iloc[:,1:]
test = pd.read_csv('/Volumes/Untitled/specsim_res/noise_second_1_100.csv').iloc[:,1:]

print(len(train))
train.dropna(how='any', inplace=True)
print(len(train))

print(len(test))
test.dropna(how='any', inplace=True)
print(len(test))

val=test.iloc[:int(2e5)]
test=test.iloc[int(2e5):]



In [ ]:
train.iloc[:10,cols['nonspec_only']]

In [ ]:
with open('/Users/jonahpoczobutt/projects/specsim_res/col_dict.pickle', 'rb') as handle:
    cols = pickle.load(handle)

cols['random_half']=list(np.random.choice(576,size=int(576/2),replace=False))
cols['random_quarter']=list(np.random.choice(576,size=int(576/4),replace=False))
cols['random_eighth']=list(np.random.choice(576,size=int(576/8),replace=False))
cols['random_tenth']=list(np.random.choice(576,size=int(576/10),replace=False))
cols['all']=list(range(576))

print(len(cols))
for i in cols:
    print(f'{i} {len(cols[i])}')

In [8]:


def zero_one_loss(preds, true):
    
    preds=np.array(preds).squeeze()
    true=np.array(true).squeeze()
    
    return sum(abs(true-preds))/len(true)

def best_model_select(models, train, val, test):
    
    model_errs = np.zeros(len(models))
    train['match']=train['match'].astype(int)
    val['match']=val['match'].astype(int)
    test['match']=test['match'].astype(int)

    best_model = None
    best_error=np.inf

    i=0
    for model in models:

        clf = sklearn.base.clone(model)
        clf.fit(train.iloc[:,:-1], train.iloc[:,-1])
        val_error = zero_one_loss(clf.predict(val.iloc[:,:-1]),val.iloc[:,-1:].to_numpy())

        if val_error < best_error:
            best_model=copy.deepcopy(clf)
            best_error = val_error

        del(clf)
        model_errs[i]=val_error
        i+=1

    return str(best_model), zero_one_loss(best_model.predict(test.iloc[:,:-1]),test.iloc[:,-1:].to_numpy()), model_errs

def best_models_by_subset(cols, train_sizes, models, train, val, test):

    res_dict=dict()
    for key, value in cols.items():
        res_dict[key]=list()

        for size in train_sizes:
    
            res_dict[key].append(best_model_select(models, train.iloc[:size,value+[-1]], val.iloc[:,value+[-1]], test.iloc[:,value+[-1]]))

            print(f'finished {key} for {size}')

    return res_dict



In [18]:

models = [
        hgbc(),
        hgbc(learning_rate=0.01),
        hgbc(learning_rate=0.5),
        hgbc(max_iter=200),
        hgbc(learning_rate=0.01, max_iter=200),
        hgbc(learning_rate=0.5, max_iter=200),
        hgbc(min_samples_leaf=10),
        hgbc(learning_rate=0.01,min_samples_leaf=10),
        hgbc(learning_rate=0.5,min_samples_leaf=10),
        hgbc(max_iter=200,min_samples_leaf=10),
        hgbc(learning_rate=0.01, max_iter=200,min_samples_leaf=10),
        hgbc(learning_rate=0.5, max_iter=200,min_samples_leaf=10),
        ]

#model, test_err, val_errs = best_model_select(models, train[:1000], val[:1000], test[:1000]) 

In [ ]:
train_sizes=[int(8e5)]
res_dict2 = best_models_by_subset(cols2, train_sizes=train_sizes, models=models, train=train, val=val, test=test )
        



In [ ]:
with open('/Users/jonahpoczobutt/projects/specsim_res/res_dict2.pickle', 'wb') as handle:
    pickle.dump(res_dict2, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
res_dict2

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(10,6))
for i in res_dict2:

    if i in res_dict2:
    #if 'all' in i or i=='nonspec_only':
        test_errors = [j[1] for j in res_dict2[i]]
        plt.scatter(train_sizes, test_errors, label=i)


plt.title('Zero One Error by Proportion of Features Included')
plt.xlabel('# of Training Points')
plt.ylabel('Zero One Error')
plt.legend(loc='best')
plt.show()




In [ ]:
import spectral_similarity

sims=list()
for i in spectral_similarity.methods_range:
    sims.append(i)
    sims.append('max_'+i)
    sims.append('min_'+i)
    sims.append('ave_'+i)
    
mod_data = datasetBuilder.create_model_dataset(target, 10, sims, 10, nonspec_features=False, spec_features=False)

In [ ]:
mod_data=mod_data.iloc[:,1:]
mod_data.columns=sims
mod_data

In [ ]:
import scipy.stats
scipy.stats.entropy([1 for i in range(10000)])

In [ ]:
import numpy as np
import plotAndOrderResults

In [ ]:
import pandas as pd
df_res=pd.read_csv("C:\\Users\\jonah\\projects\\SpecSim\\allsim_res_4.csv")

In [ ]:
plotAndOrderResults.add_evals_to_df(df_res)

In [ ]:
plotAndOrderResults.order_criterion_from_df(df_res)

In [ ]:
plotAndOrderResults.order_criterion_from_df(df_res)

In [ ]:
yool = spectral_similarity.multiple_similarity(b,a, ms2_da=0.05, methods=['max_lorentzian','reverse_lorentzian'])

In [ ]:
yool['reverse_lorentzian']

In [ ]:
a

In [ ]:
yool['reverse_entropy']

In [ ]:
match = tools.match_peaks_in_spectra(a,b, ms2_da=0.05)

In [ ]:
import math_distance

In [ ]:
math_distance._select_common_peaks(a,b)

In [ ]:
yool = a>0

In [ ]:
yool